In [1]:
import eikon as ek
import pandas as pd
from sqlalchemy import create_engine

(CVXPY) Jul 12 12:41:53 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 12 12:41:53 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.6.2534). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [2]:
with open('key.txt', 'r') as f:
    ek.set_app_key(f.readline())

In [3]:
engine = create_engine('sqlite:///GDS_IndividualProject.db', echo=False)

In [8]:
df_tickers = pd.read_excel('GDS Russell 3000 as of May 2023.xlsx')

In [9]:
# Convert ISIN to List
isin_list = df_tickers['ISIN'].tolist()

# Get Eikon identifiers
df_ric = ek.get_symbology(isin_list, from_symbol_type='ISIN', to_symbol_type='RIC', best_match=True)

df_ric.drop(columns=['error'], inplace=True)
df_ric.reset_index(inplace=True)
df_ric.rename(columns={'index': 'ISIN'}, inplace=True)

In [10]:
# Merge with tickers data
df_tickers = df_tickers.merge(df_ric, on='ISIN', how='left')

In [11]:
# Drop privately held/delisted companies
df_tickers.dropna(subset=['RIC'], inplace=True)

In [12]:
df_tickers.to_sql('tickers', con=engine, if_exists='fail', index=False)

ValueError: Table 'tickers' already exists.

In [13]:
df_intensity, err = ek.get_data(instruments=df_tickers['RIC'].tolist(), 
                      fields=['TR.CO2DirectScope1', 'TR.CO2IndirectScope2',  'TR.TotalRevenue'])

df_intensity.dropna(inplace=True)

c:\Users\Gavin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1422: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\Gavin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1441: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [14]:
df_intensity

,Instrument,"CO2 Equivalent Emissions Direct, Scope 1","CO2 Equivalent Emissions Indirect, Scope 2",Total Revenue
0,AA,17400000.0,4400000.0,12451000000.0
2,AXP,17944.0,80275.0,54389000000.0
5,BA,612000.0,756000.0,66608000000.0
7,PEP.O,3596712.0,1811114.0,86392000000.0
8,CAT,740000.0,800000.0,59427000000.0
...,...,...,...,...
2862,IR,35999.0,58812.0,5916300000.0
2867,FANG.O,1252665.0,528224.0,9643000000.0
2869,DBX.O,1053.0,37106.0,2324900000.0
2877,TROX.K,2400000.0,2000000.0,3454000000.0


In [15]:
df_intensity['Carbon Intensity'] = 1000000*(df_intensity['CO2 Equivalent Emissions Direct, Scope 1'] + df_intensity['CO2 Equivalent Emissions Indirect, Scope 2']) / df_intensity['Total Revenue']

In [23]:
df_intensity.to_sql('emissions', con=engine, if_exists='fail', index=False)

1040

In [18]:
df_esg, err = ek.get_data(instruments=df_intensity['Instrument'].tolist(), fields=[
    'TR.TRESGScore', 'TR.EnvironmentPillarScore', 'TR.SocialPillarScore', 'TR.GovernancePillarScore'])

In [25]:
df_esg.to_sql('esg', con=engine, if_exists='fail', index=False)

1040

In [20]:
df_rets, err = ek.get_data(instruments=df_intensity['Instrument'].tolist(), 
                      fields=['TR.TotalReturn.Date', 'TR.TotalReturn'], 
                        parameters={'SDate': '2017-12-31', 'EDate': '2022-12-31', 'FRQ': 'M'})

In [21]:
df_rets = df_rets.loc[df_rets['Date']!=''] # Remove rows with empty returns data (Due to not being listed at the time)
df_rets['Total Return'] = df_rets['Total Return'].astype(float)
df_rets.dropna(inplace=True)

In [28]:
df_rets.to_sql('returns', con=engine, if_exists='fail', index=False)

61318

In [29]:
random_RIC = pd.Series(df_rets['Instrument'].unique()).sample(100, random_state=42)

df_sample = df_rets.loc[df_rets['Instrument'].isin(random_RIC)].reset_index(drop=True)
df_sample['Date'] = pd.to_datetime(df_sample['Date'])
df_sample['Date'] = df_sample['Date'].dt.strftime('%Y-%m')

df_sample = df_sample.pivot(index='Date', columns='Instrument', values='Total Return')

df_sample = df_sample/100
df_sample

Instrument,ABM,ABT,ACN,ADBE.O,AFL,AKR,APTV.K,ATR,AWK,BAH,...,UAL.O,UDR,UE,UEC,USFD.K,WDAY.O,WK,WMT,ZBRA.O,ZS.O
Date,,,,,,,,,,,,,,,,,,,,,
2017-12,-0.118692,0.012418,0.034322,-0.034331,0.001597,-0.014312,-0.032394,-0.024092,-0.000765,-0.014474,...,0.064435,-0.020595,0.006341,0.330827,0.096497,-0.012233,-0.016092,0.020985,-0.059101,NaN
2018-01,0.012896,0.094362,0.049709,0.139922,0.004785,-0.102339,0.118472,0.016910,-0.090939,0.027537,...,0.006231,-0.043703,-0.082778,-0.112994,0.006264,0.178396,0.042056,0.079494,0.186513,NaN
2018-02,-0.075467,-0.029440,0.001929,0.046906,0.013603,-0.019544,-0.035168,0.022878,-0.040795,-0.026966,...,-0.000442,-0.079661,-0.076989,-0.171975,0.039216,0.056552,0.020179,-0.155629,0.121630,NaN
2018-03,-0.047782,-0.006796,-0.046643,0.033233,-0.015302,0.032721,-0.069638,0.004585,0.034904,0.020828,...,0.024782,0.059488,-0.000406,0.007692,-0.018568,0.003474,0.041758,-0.005708,0.007601,-0.149394
2018-04,-0.065316,-0.025256,-0.006156,0.025546,0.041362,-0.040650,-0.004590,0.040855,0.054182,0.023502,...,-0.027782,0.024045,-0.036534,0.152672,0.043027,-0.017859,-0.050633,-0.005732,-0.031324,0.064838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08,0.034790,-0.056872,-0.058121,-0.089437,0.043733,-0.070053,-0.109257,-0.045935,-0.040937,0.001556,...,-0.047347,-0.072934,-0.042605,0.076190,-0.027937,0.060993,0.036489,0.008116,-0.156700,0.026957
2022-09,-0.176078,-0.057379,-0.108022,-0.263068,-0.054191,-0.196367,-0.162903,-0.075674,-0.123206,-0.035005,...,-0.070837,-0.070426,-0.143179,-0.225664,-0.136512,-0.074988,0.145971,-0.021501,-0.131382,0.032216
2022-10,0.169720,0.027220,0.108323,0.157340,0.158541,0.106973,0.164429,0.047351,0.116626,0.178668,...,0.324316,-0.037229,0.058471,0.202857,0.125567,0.023650,0.000129,0.097379,0.080951,-0.062481


In [31]:
df_sector, err = ek.get_data(instruments=random_RIC.tolist(), fields=['TR.TRBCEconomicSector'])

In [38]:
df_sample.to_sql('sample', con=engine, if_exists='fail', index=True)
df_sector.to_sql('sectors', con=engine, if_exists='fail', index=False)

100

In [22]:
df_mkt_cap, err = ek.get_data(instruments=df_tickers['RIC'].tolist(), 
                              fields=['TR.CompanyMarketCap.Date', 'TR.CompanyMarketCap'],
                              parameters={'SDate': '2017-12-31', 'EDate': '2022-12-31', 'FRQ': 'M'})

c:\Users\Gavin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1422: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\Gavin\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1441: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [11]:
df_mkt_cap.fillna(method='ffill', inplace=True)
df_mkt_cap['Date'] = pd.to_datetime(df_mkt_cap['Date'])
df_mkt_cap['Date'] = df_mkt_cap['Date'].dt.strftime('%Y-%m')
df_mkt_cap.dropna(inplace=True)

df_mkt_cap = df_mkt_cap.pivot(index='Date', columns='Instrument', values='Company Market Cap')

In [13]:
df_mkt_cap.to_sql('market_cap', con=engine, if_exists='fail', index=True)

61